In [1]:
from ast import literal_eval
from os import listdir
from os.path import isfile, join
from scipy.sparse import csr_matrix, load_npz, save_npz
from tqdm import tqdm

import seaborn as sns
import datetime
import json
import numpy as np
import pandas as pd
import time
import yaml
import scipy.sparse as sparse
from ast import literal_eval

In [2]:
df_train = pd.read_csv('beer/Train.csv')

In [3]:
df_valid = pd.read_csv('beer/Valid.csv')

In [4]:
df_test = pd.read_csv('beer/Test.csv')

In [5]:
key_phrase = pd.read_csv('beer/KeyPhrases.csv')

In [6]:
df_train.head()

,beer/ABV,beer/name,beer/style,review/overall,review/text,review/timeUnix,user/profileName,Binary,UserIndex,ItemIndex,review,conca_review,keyVector,keyphrases_indices_length
0,5.5,Caldera Pale Ale,American Pale Ale (APA),4.0,Caldera- Pale Ale\t\tPicked up at Grape & Gour...,1251327677,NJpadreFan,0,1767,659,"['caldera', 'pale', 'ale', 'picked', 'grape', ...",caldera pale ale picked grape gourmet vb ligh...,"[1, 7, 12, 14, 16, 29, 31, 34, 39, 40, 42, 45]",12
1,5.5,Caldera Pale Ale,American Pale Ale (APA),2.5,12 oz. can poured into a pint glass. I really ...,1250928902,vacax,0,6159,659,"['oz', 'poured', 'pint', 'glass', 'really', 'l...",oz poured pint glass really like beer like pa...,"[1, 16, 18, 20, 24, 39, 42, 43, 45, 55]",10
2,5.5,Caldera Pale Ale,American Pale Ale (APA),4.0,A: Pours a cloudy orange with a crisp white tw...,1249866208,d0ggnate,0,3516,659,"['pours', 'cloudy', 'orange', 'crisp', 'white'...",pours cloudy orange crisp white two finger he...,"[12, 36, 39, 42]",4
3,5.5,Caldera Pale Ale,American Pale Ale (APA),4.5,Received this can through Beer Powerball. I am...,1249847121,babyhobbes,1,2893,659,"['received', 'beer', 'powerball', 'really', 's...",received beer powerball really sorry nt know ...,"[12, 42]",2
4,5.5,Caldera Pale Ale,American Pale Ale (APA),4.5,355ml can bought from Beersofeurope.com...\t\t...,1249556277,mdagnew,1,4858,659,"['bought', 'beersofeurope', 'com', 'poured', '...",bought beersofeurope com poured golden yellow...,"[1, 5, 7, 12, 14, 16, 18, 19, 26, 31, 34, 40, ...",18


In [7]:
def get_beer_df(df, sampling=False, top_user_num=6100, top_item_num=4000):
    if sampling:
        df = filter_yelp_df(df, top_user_num=top_user_num, top_item_num=top_item_num)

    rating_matrix = df_to_sparse(df, row_name='UserIndex',
                                 col_name='ItemIndex',
                                 value_name='Binary',
                                 shape=None)

    timestamp_matrix = df_to_sparse(df, row_name='UserIndex',
                                    col_name='ItemIndex',
                                    value_name='review/timeUnix',
                                    shape=None)
    return rating_matrix, timestamp_matrix

def df_to_sparse(df, row_name='UserIndex', col_name='ItemIndex', value_name='Binary',
                 shape=(6370,3668)):
    rows = df[row_name]
    cols = df[col_name]
    if value_name is not None:
        values = df[value_name]
    else:
        values = [1]*len(rows)

    return csr_matrix((values, (rows, cols)), shape=shape)

In [8]:
def time_ordered_split(rating_matrix, timestamp_matrix, ratio=[0.5, 0.2, 0.3],
                       implicit=True, remove_empty=False, threshold=4,
                       sampling=False, sampling_ratio=0.1):

    if implicit:
#         temp_rating_matrix = sparse.csr_matrix(rating_matrix.shape)
#         temp_rating_matrix[(rating_matrix > threshold).nonzero()] = 1
#         rating_matrix = temp_rating_matrix
        timestamp_matrix = timestamp_matrix.multiply(rating_matrix)

    nonzero_index = None
    
    user_num, item_num = rating_matrix.shape

    rtrain = []
    rtime = []
    rvalid = []
    rtest = []
#     item_idx_matrix = []
#     item_idx_valid = []
#     item_idx_test = []
    
    for i in tqdm(range(user_num)):
        item_indexes = rating_matrix[i].nonzero()[1]
        data = rating_matrix[i].data
        timestamp = timestamp_matrix[i].data
        num_nonzeros = len(item_indexes)
        if num_nonzeros >= 1:
            num_test = int(num_nonzeros * ratio[2])
            num_valid = int(num_nonzeros * (ratio[1] + ratio[2]))

            valid_offset = num_nonzeros - num_valid
            test_offset = num_nonzeros - num_test

            argsort = np.argsort(timestamp)
            data = data[argsort]
            item_indexes = item_indexes[argsort]
            rtrain.append([data[:valid_offset], np.full(valid_offset, i), item_indexes[:valid_offset]])
            rvalid.append([data[valid_offset:test_offset], np.full(test_offset - valid_offset, i),
                           item_indexes[valid_offset:test_offset]])
            rtest.append([data[test_offset:], np.full(num_test, i), item_indexes[test_offset:]])
            
#             item_idx_matrix.append(item_indexes[:valid_offset])
#             item_idx_valid.append(item_indexes[valid_offset:test_offset])
#             item_idx_test.append(item_indexes[test_offset:])
#         else:
#             item_idx_matrix.append([])
#             item_idx_valid.append([])
#             item_idx_test.append([])
    rtrain = np.array(rtrain)
    rvalid = np.array(rvalid)
    rtest = np.array(rtest)

    rtrain = sparse.csr_matrix((np.hstack(rtrain[:, 0]), (np.hstack(rtrain[:, 1]), np.hstack(rtrain[:, 2]))),
                               shape=rating_matrix.shape, dtype=np.float32)
    rvalid = sparse.csr_matrix((np.hstack(rvalid[:, 0]), (np.hstack(rvalid[:, 1]), np.hstack(rvalid[:, 2]))),
                               shape=rating_matrix.shape, dtype=np.float32)
    rtest = sparse.csr_matrix((np.hstack(rtest[:, 0]), (np.hstack(rtest[:, 1]), np.hstack(rtest[:, 2]))),
                              shape=rating_matrix.shape, dtype=np.float32)

    return rtrain, rvalid, rtest, timestamp_matrix

rating_matrix, timestamp_matrix = get_beer_df(df)

rtrain, rvalid, rtest, timestamp_matrix = time_ordered_split(rating_matrix, timestamp_matrix)

rating_matrix

In [9]:
def get_beer_binary(df, shape = (6370,3668)):
    rating_matrix = df_to_sparse(df, row_name='UserIndex',
                                 col_name='ItemIndex',
                                 value_name='Binary',
                                 shape=shape)
    return rating_matrix

def df_to_sparse(df, row_name='UserIndex', col_name='ItemIndex', value_name='Binary',
                 shape=(6370,3668)):
    rows = df[row_name]
    cols = df[col_name]
    if value_name is not None:
        values = df[value_name]
    else:
        values = [1]*len(rows)

    return csr_matrix((values, (rows, cols)), shape=shape)

In [18]:
rtrain = get_beer_binary(df_train)
rvalid = get_beer_binary(df_valid)
rtest = get_beer_binary(df_test)

In [17]:
rtrain

<6370x3668 sparse matrix of type '<type 'numpy.int64'>'
	with 632987 stored elements in Compressed Sparse Row format>

In [10]:
from sklearn.metrics.pairwise import cosine_similarity
def train(matrix_train):
    similarity = cosine_similarity(X=matrix_train, Y=None, dense_output=True)
    return similarity

def get_I_K(df, row_name = 'ItemIndex', shape = (3668,75)):
    rows = []
    cols = []
    vals = []
    for i in tqdm(range(df.shape[0])):
        key_vector = literal_eval(df['keyVector'][i])
        rows.extend([df[row_name][i]]*len(key_vector)) ## Item index
        cols.extend(key_vector) ## Keyword Index
#         if binary:
        vals.extend(np.array([1]*len(key_vector)))
#         else:
#             vals.extend(arr[arr.nonzero()])    
    return csr_matrix((vals, (rows, cols)), shape=shape)


def predict(matrix_train, k, similarity, item_similarity_en = False):
    prediction_scores = []
    
    if item_similarity_en:
        matrix_train = matrix_train.transpose()
        
    for user_index in tqdm(range(matrix_train.shape[0])):
        # Get user u's prediction scores for all items
        vector_u = similarity[user_index]

        # Get closest K neighbors excluding user u self
        similar_users = vector_u.argsort()[::-1][1:k+1]
        # Get neighbors similarity weights and ratings
        similar_users_weights = similarity[user_index][similar_users]
        similar_users_ratings = matrix_train[similar_users].toarray()

        prediction_scores_u = similar_users_ratings * similar_users_weights[:, np.newaxis]

        prediction_scores.append(np.sum(prediction_scores_u, axis=0))
    res = np.array(prediction_scores)
    if item_similarity_en:
        res = res.transpose()
    return res

def prediction(prediction_score, topK, matrix_Train):

    prediction = []

    for user_index in tqdm(range(matrix_Train.shape[0])):
        vector_u = prediction_score[user_index]
        vector_train = matrix_Train[user_index]
        if len(vector_train.nonzero()[0]) > 0:
            vector_predict = sub_routine(vector_u, vector_train, topK=topK)
        else:
            vector_predict = np.zeros(topK, dtype=np.float32)

        prediction.append(vector_predict)

    return np.vstack(prediction)


def sub_routine(vector_u, vector_train, topK=500):

    train_index = vector_train.nonzero()[1]

    vector_u = vector_u

    candidate_index = np.argpartition(-vector_u, topK+len(train_index))[:topK+len(train_index)]
    vector_u = candidate_index[vector_u[candidate_index].argsort()[::-1]]
    vector_u = np.delete(vector_u, np.isin(vector_u, train_index).nonzero()[0])

    return vector_u[:topK]



In [116]:
I_K

<3668x75 sparse matrix of type '<type 'numpy.int32'>'
	with 213696 stored elements in Compressed Sparse Row format>

In [117]:
save_npz('beer/I_K',I_K)

In [121]:
U_K = get_I_K(df_train, row_name = 'UserIndex', shape = (6370,75))

100%|███████████████████████████████████████████████████████████████████████| 641773/641773 [00:32<00:00, 20000.41it/s]


In [123]:
save_npz('beer/U_K',U_K)

In [ ]:
get

In [114]:
similarity[0]

array([0.        , 0.82145752, 0.35454307, ..., 0.92327245, 0.36836629,
       0.34408779])

In [115]:
rtrain.dot(similarity[0])

array([17.123164  , 18.35822566,  4.25679072, ...,  5.20957769,
        4.39997092,  9.04351073])

In [112]:
similar_users

array([2759,  289, 3639,  184, 2744, 2809, 2691, 2028, 3015, 1037, 2468,
       3642,  286, 3472, 2707, 2829, 2806,  474, 2923, 1800, 3214, 3621,
       1782, 2952, 2132, 1870,  453,  345,  452, 3654, 2469, 2447, 1959,
       1838,  999, 2354,  544, 2061,   19, 2607,  367, 2701, 3149, 1443,
       1023, 2260, 2306, 3387, 1504,  449, 3574, 3391, 2865, 2761, 3243,
        457, 2931, 2172, 1460, 1369,  486, 3074,   29, 2232, 2121, 2776,
       2253, 1844, 3506, 3497, 2071, 3313, 2876,  640, 1390, 2749,  500,
        692,  477, 1599, 1633, 1021,  305, 2053,  469, 2138, 3601, 2543,
        287, 1249, 1410, 2244,  371, 3228, 3092,  876, 2215, 1271,  825,
       2666], dtype=int64)

In [98]:
similarity = I_I
user_index = 0
k = 100
matrix_train = rtrain
vector_u = similarity[user_index]
similar_users = vector_u.argsort()[::-1][1:k+1]
similar_users_weights = similarity[user_index][similar_users]
similar_users_ratings = matrix_train[similar_users].toarray()
prediction_scores_u = similar_users_ratings * similar_users_weights[:, np.newaxis]

### Evaluation

In [11]:
def recallk(vector_true_dense, hits, **unused):
    hits = len(hits.nonzero()[0])
    return float(hits)/len(vector_true_dense)


def precisionk(vector_predict, hits, **unused):
    hits = len(hits.nonzero()[0])
    return float(hits)/len(vector_predict)


def average_precisionk(vector_predict, hits, **unused):
    precisions = np.cumsum(hits, dtype=np.float32)/range(1, len(vector_predict)+1)
    return np.mean(precisions)


def r_precision(vector_true_dense, vector_predict, **unused):
    vector_predict_short = vector_predict[:len(vector_true_dense)]
    hits = len(np.isin(vector_predict_short, vector_true_dense).nonzero()[0])
    return float(hits)/len(vector_true_dense)


def _dcg_support(size):
    arr = np.arange(1, size+1)+1
    return 1./np.log2(arr)


def ndcg(vector_true_dense, vector_predict, hits):
    idcg = np.sum(_dcg_support(len(vector_true_dense)))
    dcg_base = _dcg_support(len(vector_predict))
    dcg_base[np.logical_not(hits)] = 0
    dcg = np.sum(dcg_base)
    return dcg/idcg


def click(hits, **unused):
    first_hit = next((i for i, x in enumerate(hits) if x), None)
    if first_hit is None:
        return 5
    else:
        return first_hit/10


def evaluate(matrix_Predict, matrix_Test, metric_names =['R-Precision', 'NDCG', 'Precision', 'Recall', 'MAP'], atK = [5, 10, 15, 20, 50], analytical=False):
    """
    :param matrix_U: Latent representations of users, for LRecs it is RQ, for ALSs it is U
    :param matrix_V: Latent representations of items, for LRecs it is Q, for ALSs it is V
    :param matrix_Train: Rating matrix for training, features.
    :param matrix_Test: Rating matrix for evaluation, true labels.
    :param k: Top K retrieval
    :param metric_names: Evaluation metrics
    :return:
    """
    global_metrics = {
        "R-Precision": r_precision,
        "NDCG": ndcg,
        "Clicks": click
    }

    local_metrics = {
        "Precision": precisionk,
        "Recall": recallk,
        "MAP": average_precisionk
    }

    output = dict()

    num_users = matrix_Predict.shape[0]

    for k in atK:

        local_metric_names = list(set(metric_names).intersection(local_metrics.keys()))
        results = {name: [] for name in local_metric_names}
        topK_Predict = matrix_Predict[:, :k]

        for user_index in tqdm(range(topK_Predict.shape[0])):
            vector_predict = topK_Predict[user_index]
            if len(vector_predict.nonzero()[0]) > 0:
                vector_true = matrix_Test[user_index]
                vector_true_dense = vector_true.nonzero()[1]
                hits = np.isin(vector_predict, vector_true_dense)

                if vector_true_dense.size > 0:
                    for name in local_metric_names:
                        results[name].append(local_metrics[name](vector_true_dense=vector_true_dense,
                                                                 vector_predict=vector_predict,
                                                                 hits=hits))

        results_summary = dict()
        if analytical:
            for name in local_metric_names:
                results_summary['{0}@{1}'.format(name, k)] = results[name]
        else:
            for name in local_metric_names:
                results_summary['{0}@{1}'.format(name, k)] = (np.average(results[name]),
                                                              1.96*np.std(results[name])/np.sqrt(num_users))
        output.update(results_summary)

    global_metric_names = list(set(metric_names).intersection(global_metrics.keys()))
    results = {name: [] for name in global_metric_names}

    topK_Predict = matrix_Predict[:]

    for user_index in tqdm(range(topK_Predict.shape[0])):
        vector_predict = topK_Predict[user_index]

        if len(vector_predict.nonzero()[0]) > 0:
            vector_true = matrix_Test[user_index]
            vector_true_dense = vector_true.nonzero()[1]
            hits = np.isin(vector_predict, vector_true_dense)

            # if user_index == 1:
            #     import ipdb;
            #     ipdb.set_trace()

            if vector_true_dense.size > 0:
                for name in global_metric_names:
                    results[name].append(global_metrics[name](vector_true_dense=vector_true_dense,
                                                              vector_predict=vector_predict,
                                                              hits=hits))

    results_summary = dict()
    if analytical:
        for name in global_metric_names:
            results_summary[name] = results[name]
    else:
        for name in global_metric_names:
            results_summary[name] = (np.average(results[name]), 1.96*np.std(results[name])/np.sqrt(num_users))
    output.update(results_summary)

    return output


## User-Item 

In [38]:
similarity = train(rtrain)
user_item_prediction_score = predict(rtrain, 100, similarity, item_similarity_en= False)
user_item_predict = prediction(user_item_prediction_score, 50, rtrain)

100%|████████████████████████████████████████████████████████████████████████████| 6370/6370 [00:01<00:00, 3477.07it/s]


In [39]:
user_item_res = evaluate(user_item_predict, rvalid)

100%|████████████████████████████████████████████████████████████████████████████| 6370/6370 [00:01<00:00, 5128.82it/s]


In [40]:
user_item_res

{'MAP@10': (0.045144826330695706, 0.002354497505081906),
 'MAP@15': (0.042265773714731864, 0.0019824373381772792),
 'MAP@20': (0.040129338606794795, 0.0017567965943763916),
 'MAP@5': (0.04956399929959726, 0.003133599222210933),
 'MAP@50': (0.032943936395371624, 0.0012118312514037895),
 'NDCG': (0.10093614993594359, 0.003179003332122599),
 'Precision@10': (0.03883733146559272, 0.0017160204191020419),
 'Precision@15': (0.035171890503706296, 0.0014211672707212524),
 'Precision@20': (0.032708807564349504, 0.0012423089296096815),
 'Precision@5': (0.044440553318157945, 0.002423972508292176),
 'Precision@50': (0.024941341271230957, 0.0008480003467919996),
 'R-Precision': (0.03997492992806294, 0.002317948843344031),
 'Recall@10': (0.06845154159235434, 0.0040585896263153525),
 'Recall@15': (0.09096277675804605, 0.004630203266326043),
 'Recall@20': (0.11247720176840045, 0.005101760514228912),
 'Recall@5': (0.040952551758442035, 0.0032340960406314727),
 'Recall@50': (0.19980217392786573, 0.006326

## Item-item similarity 

In [26]:
I_K = get_I_K(df_train, shape = (3668, 75))

100%|███████████████████████████████████████████████████████████████████████| 641773/641773 [00:32<00:00, 19846.40it/s]


In [27]:
I_K

<3668x75 sparse matrix of type '<type 'numpy.int32'>'
	with 213696 stored elements in Compressed Sparse Row format>

In [28]:
I_I = train(I_K)

In [29]:
item_based_prediction_score = predict(rtrain, 100, I_I, item_similarity_en= True)
item_based_predict = prediction(item_based_prediction_score, 50, rtrain)
item_based_res = evaluate(item_based_predict, rvalid)

100%|████████████████████████████████████████████████████████████████████████████| 6370/6370 [00:01<00:00, 5178.86it/s]


In [32]:
I_I.shape

(3668L, 3668L)

In [30]:
item_based_res

{'MAP@10': (0.012257297946319131, 0.0012345550264786004),
 'MAP@15': (0.011561416462484578, 0.0010299391586485126),
 'MAP@20': (0.011172197911233655, 0.0009052524737597343),
 'MAP@5': (0.013544037821747503, 0.0016795407276965935),
 'MAP@50': (0.009851991690077999, 0.0006024329263627383),
 'NDCG': (0.03036794781815371, 0.0017408316332245687),
 'Precision@10': (0.010646121519873928, 0.0008871305915328645),
 'Precision@15': (0.010085799334617406, 0.0007331559760569813),
 'Precision@20': (0.009884433549290843, 0.0006363352616858814),
 'Precision@5': (0.012081947119593768, 0.0012571530710625744),
 'Precision@50': (0.00833129049203292, 0.0004178737966964705),
 'R-Precision': (0.010566064486017419, 0.0010758446631350643),
 'Recall@10': (0.018115843733737758, 0.0022331758885910623),
 'Recall@15': (0.025011070563634115, 0.0026040777674045806),
 'Recall@20': (0.0315957917507434, 0.002862300095017566),
 'Recall@5': (0.00998045633316055, 0.0015656795588398769),
 'Recall@50': (0.06454957599052848, 

## User-User similarity

In [58]:
U_K = get_I_K(df_train, row_name = 'UserIndex', shape = (6370, 75))
U_U = train(U_K)
user_based_prediction_score = predict(rtrain, 100, U_U, item_similarity_en= False)
user_based_predict = prediction(user_based_prediction_score, 50, rtrain)
user_based_res = evaluate(user_based_predict, rvalid)



  0%|                                                                                       | 0/641773 [00:00<?, ?it/s]

  0%|▏                                                                        | 1669/641773 [00:00<00:43, 14769.92it/s]

  1%|▍                                                                        | 3505/641773 [00:00<00:40, 15690.33it/s]

  1%|▋                                                                        | 5809/641773 [00:00<00:37, 16782.07it/s]

  1%|▉                                                                        | 7817/641773 [00:00<00:36, 17605.38it/s]

  2%|█                                                                        | 9775/641773 [00:00<00:34, 18104.28it/s]

  2%|█▎                                                                      | 11731/641773 [00:00<00:34, 18465.28it/s]

  2%|█▌                                                                      | 13472/641773 [00:00<00:34, 18079.02it/s]

  2%|█▋                       

 42%|█████████████████████████████▌                                         | 267275/641773 [00:14<00:19, 19181.08it/s]

 42%|█████████████████████████████▊                                         | 269520/641773 [00:14<00:19, 19331.83it/s]

 42%|██████████████████████████████                                         | 271459/641773 [00:14<00:19, 19291.49it/s]

 43%|██████████████████████████████▎                                        | 273729/641773 [00:14<00:18, 19423.63it/s]

 43%|██████████████████████████████▌                                        | 275701/641773 [00:14<00:18, 19453.87it/s]

 43%|██████████████████████████████▋                                        | 277649/641773 [00:14<00:18, 19461.69it/s]

 44%|██████████████████████████████▉                                        | 279947/641773 [00:14<00:18, 19565.00it/s]

 44%|███████████████████████████████▏                                       | 282060/641773 [00:14<00:18, 19952.91it/s]

 44%|███████████████████████████

 84%|███████████████████████████████████████████████████████████▍           | 537772/641773 [00:28<00:05, 19241.32it/s]

 84%|███████████████████████████████████████████████████████████▋           | 539747/641773 [00:28<00:05, 19277.59it/s]

 84%|███████████████████████████████████████████████████████████▉           | 541684/641773 [00:28<00:05, 19305.21it/s]

 85%|████████████████████████████████████████████████████████████▏          | 543880/641773 [00:28<00:05, 19292.57it/s]

 85%|████████████████████████████████████████████████████████████▍          | 545876/641773 [00:28<00:04, 19374.57it/s]

 85%|████████████████████████████████████████████████████████████▋          | 548161/641773 [00:28<00:04, 19520.46it/s]

 86%|████████████████████████████████████████████████████████████▊          | 550238/641773 [00:28<00:04, 19006.08it/s]

 86%|█████████████████████████████████████████████████████████████          | 552215/641773 [00:28<00:04, 19173.05it/s]

 86%|███████████████████████████

 40%|██████████████████████████████▋                                              | 2538/6370 [00:08<00:13, 291.96it/s]

 40%|███████████████████████████████                                              | 2568/6370 [00:08<00:12, 294.32it/s]

 41%|███████████████████████████████▍                                             | 2598/6370 [00:08<00:12, 294.26it/s]

 41%|███████████████████████████████▊                                             | 2631/6370 [00:08<00:12, 293.59it/s]

 42%|████████████████████████████████▏                                            | 2661/6370 [00:09<00:12, 293.75it/s]

 42%|████████████████████████████████▌                                            | 2695/6370 [00:09<00:12, 294.32it/s]

 43%|████████████████████████████████▉                                            | 2726/6370 [00:09<00:12, 297.99it/s]

 43%|█████████████████████████████████▎                                           | 2756/6370 [00:09<00:12, 297.70it/s]

 44%|███████████████████████████

  0%|                                                                                         | 0/6370 [00:00<?, ?it/s]

  5%|████                                                                         | 339/6370 [00:00<00:02, 2922.42it/s]

 11%|████████▍                                                                    | 693/6370 [00:00<00:01, 2960.04it/s]

 17%|████████████▊                                                               | 1077/6370 [00:00<00:01, 3064.41it/s]

 22%|████████████████▉                                                           | 1419/6370 [00:00<00:01, 3154.32it/s]

 28%|████████████████████▉                                                       | 1754/6370 [00:00<00:01, 3201.38it/s]

 33%|█████████████████████████                                                   | 2099/6370 [00:00<00:01, 3138.18it/s]

 38%|████████████████████████████▊                                               | 2419/6370 [00:00<00:01, 3156.47it/s]

 43%|███████████████████████████

In [59]:
user_based_res

{'MAP@10': (0.031344898316532005, 0.0019419718247370961),
 'MAP@15': (0.029866232135536987, 0.0016325577177255409),
 'MAP@20': (0.028656128642907317, 0.0014423906282694256),
 'MAP@5': (0.033376524835113525, 0.002590995553761898),
 'MAP@50': (0.024320038944187366, 0.0009847994668677619),
 'NDCG': (0.07721138347289283, 0.0028333472139054983),
 'Precision@10': (0.028383820696900722, 0.0014345802902826993),
 'Precision@15': (0.02613669526644487, 0.0011749008756399859),
 'Precision@20': (0.024505340570828225, 0.001029564088716077),
 'Precision@5': (0.030887760462265802, 0.002026027617556462),
 'Precision@50': (0.019355629486955, 0.0006955690047862636),
 'R-Precision': (0.028883717093828115, 0.002038446870856928),
 'Recall@10': (0.052008110922374604, 0.0036721612954724284),
 'Recall@15': (0.07063100178823213, 0.004204104035231979),
 'Recall@20': (0.08660429125621157, 0.004636511047404981),
 'Recall@5': (0.029146689761132605, 0.002806545567276444),
 'Recall@50': (0.15816837853808208, 0.005809

## Tunning

In [12]:
def hyper_parameter_tuning(rtrain, rvalid, save_path, I_K = None, item_based = False):
    table_path = 'tables/'
    try:
        df = load_dataframe_csv(table_path, save_path)
    except:
        df = pd.DataFrame(columns=['model', 'k', 'topK'])

    num_user = rtrain.shape[0]

    for algorithm in ['user_item']:
        for k in [10, 50, 100, 200,300]:
#         for k in [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 150, 200, 250, 300, 350, 400, 450, 500, 550, 600, 650, 700, 750, 800, 850, 900, 950, 1000]:

#                     format = "model: {}, k: {}"
#                     progress.section(format.format(algorithm, k))

#                     progress.subsection("Training")
#                     model = params['models'][algorithm]()
#                     model.train(train)
                    if item_based:
                        similarity = train(I_K)
                    else:
                        similarity = train(rtrain)
                    user_item_prediction_score = predict(rtrain, k, similarity, item_similarity_en= item_based)
                    user_item_predict = prediction(user_item_prediction_score, 50, rtrain)
                    result = evaluate(user_item_predict, rvalid)
#                     progress.subsection("Prediction")
#                     prediction_score = model.predict(train, k=k)

#                     prediction = predict(prediction_score=prediction_score,
#                                          topK=params['topK'][-1],
#                                          matrix_Train=train)

#                     progress.subsection("Evaluation")
#                     result = evaluate(prediction, validation, params['metric'], params['topK'])

                    result_dict = {'model': algorithm, 'k': k}

                    for name in result.keys():
                        result_dict[name] = [round(result[name][0], 4), round(result[name][1], 4)]

                    df = df.append(result_dict, ignore_index=True)

                    save_dataframe_csv(df, table_path, save_path)
                    
def save_dataframe_csv(df, path, name):
    df.to_csv(path+name, index=False)

In [28]:
hyper_parameter_tuning(rtrain, rvalid, 'item_based_knn_tunning.csv', I_K = I_K, item_based= True)

100%|████████████████████████████████████████████████████████████████████████████| 6370/6370 [00:00<00:00, 7133.26it/s]


In [132]:
user_item_res.keys()

['MAP@5',
 'Precision@10',
 'Recall@20',
 'Precision@15',
 'MAP@10',
 'R-Precision',
 'Recall@50',
 'Precision@50',
 'MAP@50',
 'Precision@20',
 'MAP@20',
 'NDCG',
 'MAP@15',
 'Recall@10',
 'Recall@5',
 'Recall@15',
 'Precision@5']

In [30]:
a = I_K[0].toarray()

In [31]:
a[a.nonzero()]

array([10, 11,  6,  1,  2,  7,  1,  6,  2,  1,  5,  3,  2,  6,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  8, 10,  2,  1,  8,  1,  7,  1,  2,
        1,  1,  1,  1])

In [35]:
np.argsort(I_I[0])

array([3157, 3571, 1302, ...,  184, 3472,    0], dtype=int64)

In [37]:
I_I[0][3472]

0.9486380093980652

## Combined

Need to change I_K in order for this to work.


In [38]:
## Utility functions
def add_two_matrix(ratio, U_I_matrix,I_K_matrix, shape = (3906, 3000+5791)):
    # ratio determine Keywords/User in the matrix
    rows = []
    cols = []
    datas = []
    I_U_matrix = U_I_matrix.transpose()
    
    for i in tqdm(range(I_K_matrix.shape[0])):
        nonzero1 = I_K_matrix[i].nonzero()
        nonzero2 = I_U_matrix[i].nonzero()
        row = [i]*(len(nonzero1[1])+len(nonzero2[1]))
        col = nonzero1[1].tolist()+ nonzero2[1].tolist()
        data = [ratio]*len(nonzero1[1])+[1-ratio]*len(nonzero2[1]) # Binary representation of I-K/U matrix
        
        rows.extend(row)
        cols.extend(col)
        datas.extend(data)
    return csr_matrix( (datas,(rows,cols)), shape=shape )

def transfer_to_implicit(rating_matrix, threshold = 0):
    temp_rating_matrix = sparse.csr_matrix(rating_matrix.shape)
    temp_rating_matrix[(rating_matrix > threshold).nonzero()] = 1
    rating_matrix = temp_rating_matrix
    return rating_matrix

In [39]:
rating_matrix

<6370x3668 sparse matrix of type '<type 'numpy.int64'>'
	with 159829 stored elements in Compressed Sparse Row format>

In [40]:
I_K_U = add_two_matrix(0.3, rtrain, I_K, shape = (3668, 75+6370))
I_I_combined = train(I_K_U)
combined_prediction_score = predict(rtrain, 200, I_I_combined, item_similarity_en= True)

100%|██████████████████████████████████████████████████████████████████████████████| 3668/3668 [01:20<00:00, 45.36it/s]


NameError: name 'rtrain_implicit' is not defined

In [41]:
combined_predict = prediction(combined_prediction_score, 50, rtrain)

100%|████████████████████████████████████████████████████████████████████████████| 6370/6370 [00:03<00:00, 1755.79it/s]


In [42]:
combined_res = evaluate(combined_predict, rvalid)

100%|████████████████████████████████████████████████████████████████████████████| 6370/6370 [00:01<00:00, 4530.58it/s]


In [43]:
combined_res

{'MAP@10': (0.00040679361272886452, 0.00022115178008677984),
 'MAP@15': (0.00039973426963834638, 0.00017892315194170376),
 'MAP@20': (0.00039346429045003002, 0.00015427611018432321),
 'MAP@5': (0.00045163868904876095, 0.00031571304560911486),
 'MAP@50': (0.00037014980068845999, 9.535978403503597e-05),
 'NDCG': (0.0036179761762986391, 0.00076182015834055656),
 'Precision@10': (0.0003597122302158274, 0.00014702170123510666),
 'Precision@15': (0.00039968025579536375, 0.00012638377198138698),
 'Precision@20': (0.0003597122302158274, 0.00010377221875087773),
 'Precision@5': (0.0003597122302158274, 0.00020810765275028895),
 'Precision@50': (0.00041966426858513193, 7.039590865693624e-05),
 'R-Precision': (0.00029976019184652276, 0.00030055787611668724),
 'Recall@10': (0.0021482813749000799, 0.00098177354787917716),
 'Recall@15': (0.0038469224620303753, 0.0013458877288426931),
 'Recall@20': (0.0042665867306155075, 0.0013835504455394737),
 'Recall@5': (0.0010991207034372502, 0.00070099255937069

1. Try if I can find similar performance with already generated Rtrain/Rval/Rtest
1. Try if I_K beat U_I with large dataset
2. Try change I_K to implicit and test both I_K and I_U_K 
2. Try tune I_K even more

## Get larger Keyphrase 

In [78]:
#Stemming and Lemmatisation
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
lem = WordNetLemmatizer()
stem = PorterStemmer()
word = 'inversely'
print ('stemming:', stem.stem(word))
print ('lemmatization:', lem.lemmatize(word, "v"))
import re 
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import RegexpTokenizer
from nltk.stem.wordnet import WordNetLemmatizer

('stemming:', u'invers')
('lemmatization:', 'inversely')


In [79]:
stop_words = set(stopwords.words("english"))

In [80]:
def get_corpus(df, review_name = 'conca_review'):
    corpus = []
    for i in tqdm(range(len(df))):
        corpus.append(df[review_name][i])
    return corpus

In [81]:
# Get corpus and CountVector
from sklearn.feature_extraction.text import CountVectorizer
import re

In [83]:
corpus = get_corpus(df_train)

100%|███████████████████████████████████████████████████████████████████████| 641773/641773 [00:07<00:00, 88605.96it/s]


In [24]:
rtrain = load_npz('beer/Rtrain.npz')

In [28]:
rvalid = load_npz('beer/Rvalid.npz')

In [139]:
cv=CountVectorizer(max_df=0.9,stop_words=stop_words, max_features=1500, ngram_range=(1,1))
X1500=cv.fit_transform(corpus)

In [91]:
X

<641773x75 sparse matrix of type '<type 'numpy.int64'>'
	with 13057442 stored elements in Compressed Sparse Row format>

In [88]:
X750

<641773x750 sparse matrix of type '<type 'numpy.int64'>'
	with 29679349 stored elements in Compressed Sparse Row format>

In [110]:
X5000

<641773x5000 sparse matrix of type '<type 'numpy.int64'>'
	with 36884141 stored elements in Compressed Sparse Row format>

In [140]:
X1500

<641773x1500 sparse matrix of type '<type 'numpy.int64'>'
	with 33317676 stored elements in Compressed Sparse Row format>

In [22]:
def get_I_K_fromX(df, X, row_name = 'business_num_id', binary = True, shape = (121994,6000)):
    rows = []
    cols = []
    vals = []
    for i in tqdm(range(X.shape[0])):
        arr = X[i].toarray() 
        nonzero_element = arr.nonzero()[1]  # Get nonzero element in each line
        length_of_nonzero = len(nonzero_element) 
        # df[row_name][i] is the item idex
        rows.extend(np.array([df[row_name][i]]*length_of_nonzero)) ## Item index
        cols.extend(nonzero_element) ## Keyword Index
        if binary:
            vals.extend(np.array([1]*length_of_nonzero))
        else:
            vals.extend(arr[arr.nonzero()])    
    return csr_matrix((vals, (rows, cols)), shape=shape)

In [141]:
I_K1500 = get_I_K_fromX(df_train,X1500, binary=False, row_name='ItemIndex',shape = (3668,5000))

100%|████████████████████████████████████████████████████████████████████████| 641773/641773 [01:31<00:00, 7046.95it/s]


In [113]:
save_npz('I_K5000', I_K5000)

In [114]:
save_npz('I_K750', I_K750)

In [ ]:
save_npz('I_K1500', I_K750)

In [96]:
I_K750

<3668x750 sparse matrix of type '<type 'numpy.int64'>'
	with 2258425 stored elements in Compressed Sparse Row format>

In [115]:
I_K5000

<3668x5000 sparse matrix of type '<type 'numpy.int64'>'
	with 5936593 stored elements in Compressed Sparse Row format>

In [142]:
I_K1500

<3668x5000 sparse matrix of type '<type 'numpy.int64'>'
	with 3651979 stored elements in Compressed Sparse Row format>

In [143]:
U_K1500 = get_I_K_fromX(df_train,X1500, binary=False, row_name='UserIndex',shape = (6370,5000))

100%|████████████████████████████████████████████████████████████████████████| 641773/641773 [01:28<00:00, 7254.05it/s]


In [117]:
save_npz('U_K5000', U_K5000)

In [100]:
U_K750

<6370x3000 sparse matrix of type '<type 'numpy.int64'>'
	with 2820970 stored elements in Compressed Sparse Row format>

In [118]:
U_K5000

<6370x5000 sparse matrix of type '<type 'numpy.int64'>'
	with 5856594 stored elements in Compressed Sparse Row format>

In [144]:
U_K1500

<6370x5000 sparse matrix of type '<type 'numpy.int64'>'
	with 4050822 stored elements in Compressed Sparse Row format>

In [47]:
I_I = train(I_K)
item_based_prediction_score = predict(rtrain, 100, I_I, item_similarity_en= True)
item_based_predict = prediction(item_based_prediction_score, 50, rtrain)
item_based_res3000 = evaluate(item_based_predict, rvalid)

100%|████████████████████████████████████████████████████████████████████████████| 6370/6370 [00:01<00:00, 5063.59it/s]


In [32]:
item_based_res

{'MAP@10': (0.012257297946319131, 0.0012345550264786004),
 'MAP@15': (0.011561416462484578, 0.0010299391586485126),
 'MAP@20': (0.011172197911233655, 0.0009052524737597343),
 'MAP@5': (0.013544037821747503, 0.0016795407276965935),
 'MAP@50': (0.009851991690077999, 0.0006024329263627383),
 'NDCG': (0.03036794781815371, 0.0017408316332245687),
 'Precision@10': (0.010646121519873928, 0.0008871305915328645),
 'Precision@15': (0.010085799334617406, 0.0007331559760569813),
 'Precision@20': (0.009884433549290843, 0.0006363352616858814),
 'Precision@5': (0.012081947119593768, 0.0012571530710625744),
 'Precision@50': (0.00833129049203292, 0.0004178737966964705),
 'R-Precision': (0.010566064486017419, 0.0010758446631350643),
 'Recall@10': (0.018115843733737758, 0.0022331758885910623),
 'Recall@15': (0.025011070563634115, 0.0026040777674045806),
 'Recall@20': (0.0315957917507434, 0.002862300095017566),
 'Recall@5': (0.00998045633316055, 0.0015656795588398769),
 'Recall@50': (0.06454957599052848, 

In [31]:
item_based_res75

{'MAP@10': (0.017991046240477163, 0.0014869876595738748),
 'MAP@15': (0.016964206276936098, 0.0012471364615973357),
 'MAP@20': (0.016234061373556784, 0.0011011717453250388),
 'MAP@5': (0.01941166170548065, 0.0019754550187593297),
 'MAP@50': (0.013945410779668136, 0.0007453846574120812),
 'NDCG': (0.04400215037660896, 0.0022644785370570767),
 'Precision@10': (0.015759061460339695, 0.0010866355059710494),
 'Precision@15': (0.014521683301231542, 0.0008866448854694443),
 'Precision@20': (0.0138329539485204, 0.0007790373921797244),
 'Precision@5': (0.01765014883558046, 0.0015290707020443447),
 'Precision@50': (0.011465592715811592, 0.0005204675810102093),
 'R-Precision': (0.016551682800371528, 0.0016000139316329407),
 'Recall@10': (0.02846431347298235, 0.0028660771861643637),
 'Recall@15': (0.03801597457814076, 0.003254958335878828),
 'Recall@20': (0.04662586317584471, 0.0035250217486709275),
 'Recall@5': (0.016572431721391685, 0.002192128277038121),
 'Recall@50': (0.0911042557968805, 0.004

In [37]:
item_based_res750

{'MAP@10': (0.017619179639403768, 0.0015029890664105914),
 'MAP@15': (0.016772514015553762, 0.0012614256488190814),
 'MAP@20': (0.016052700720240835, 0.0011100431225007932),
 'MAP@5': (0.019039280919862255, 0.0019983608543342104),
 'MAP@50': (0.013668076519028023, 0.0007372115645110378),
 'NDCG': (0.04287477126452545, 0.0021651516647857357),
 'Precision@10': (0.01574155139205043, 0.001092119625611785),
 'Precision@15': (0.014603396953248119, 0.0008850400505197317),
 'Precision@20': (0.013622833129049203, 0.0007686628521489984),
 'Precision@5': (0.01751006828926633, 0.0015465170968568228),
 'Precision@50': (0.011062861145158465, 0.0005040812076977075),
 'R-Precision': (0.01609240732055417, 0.0015108287103228043),
 'Recall@10': (0.02694689094229889, 0.0026661888631985898),
 'Recall@15': (0.037594731540320246, 0.0031285869850064423),
 'Recall@20': (0.04615567499698252, 0.003447128886913384),
 'Recall@5': (0.016042986947980422, 0.0020854932682390103),
 'Recall@50': (0.09025942884761218, 0.

In [48]:
item_based_res3000

{'MAP@10': (0.01976801938892641, 0.0015590097720437385),
 'MAP@15': (0.018514711486170076, 0.001305560652211188),
 'MAP@20': (0.017557230326300577, 0.0011499858053791272),
 'MAP@5': (0.02162960368878772, 0.002068661669941772),
 'MAP@50': (0.014426404493735662, 0.0007621976719994915),
 'NDCG': (0.04473904173034337, 0.002223276956805617),
 'Precision@10': (0.017089826650323936, 0.00112719688733466),
 'Precision@15': (0.015560614019728011, 0.000913971411186608),
 'Precision@20': (0.01413938014358256, 0.000782320862125058),
 'Precision@5': (0.019926457713185082, 0.0016213000317906318),
 'Precision@50': (0.011265977937313956, 0.0005112883078370813),
 'R-Precision': (0.017031337131251996, 0.00151404383743766),
 'Recall@10': (0.030571579150129348, 0.0028730616511561356),
 'Recall@15': (0.040335757248167084, 0.0032529199387760213),
 'Recall@20': (0.048177782683731796, 0.0035565708122178357),
 'Recall@5': (0.018182772590266613, 0.0022786123787694305),
 'Recall@50': (0.09160603208786464, 0.00471

In [41]:
user_item_res

{'MAP@10': (0.045144826330695706, 0.002354497505081906),
 'MAP@15': (0.042265773714731864, 0.0019824373381772792),
 'MAP@20': (0.040129338606794795, 0.0017567965943763916),
 'MAP@5': (0.04956399929959726, 0.003133599222210933),
 'MAP@50': (0.032943936395371624, 0.0012118312514037895),
 'NDCG': (0.10093614993594359, 0.003179003332122599),
 'Precision@10': (0.03883733146559272, 0.0017160204191020419),
 'Precision@15': (0.035171890503706296, 0.0014211672707212524),
 'Precision@20': (0.032708807564349504, 0.0012423089296096815),
 'Precision@5': (0.044440553318157945, 0.002423972508292176),
 'Precision@50': (0.024941341271230957, 0.0008480003467919996),
 'R-Precision': (0.03997492992806294, 0.002317948843344031),
 'Recall@10': (0.06845154159235434, 0.0040585896263153525),
 'Recall@15': (0.09096277675804605, 0.004630203266326043),
 'Recall@20': (0.11247720176840045, 0.005101760514228912),
 'Recall@5': (0.040952551758442035, 0.0032340960406314727),
 'Recall@50': (0.19980217392786573, 0.006326

In [49]:
rtrain

<6370x3668 sparse matrix of type '<type 'numpy.float32'>'
	with 167511 stored elements in Compressed Sparse Row format>

In [68]:
U_K = get_I_K_fromX(df_train,X75, row_name='UserIndex',shape = (6370,3000))

100%|████████████████████████████████████████████████████████████████████████| 641773/641773 [01:06<00:00, 9621.93it/s]


In [69]:
U_U = train(U_K)
user_based_prediction_score = predict(rtrain, 100, U_U, item_similarity_en= False)
user_based_predict = prediction(user_based_prediction_score, 50, rtrain)
user_based_res75 = evaluate(user_based_predict, rvalid)

100%|████████████████████████████████████████████████████████████████████████████| 6370/6370 [00:01<00:00, 5191.52it/s]


In [70]:
user_based_res75

{'MAP@10': (0.0275064967912105, 0.0018156430302940286),
 'MAP@15': (0.026319282512593665, 0.001530283656584368),
 'MAP@20': (0.025362228176172386, 0.0013588372235038583),
 'MAP@5': (0.02921788361641277, 0.0024008746050957707),
 'MAP@50': (0.022092810723263818, 0.0009415496598072742),
 'NDCG': (0.06959200705100783, 0.0025790311726682498),
 'Precision@10': (0.024864296970758187, 0.0013472834954877417),
 'Precision@15': (0.023195003793848135, 0.001109670042827332),
 'Precision@20': (0.022132726317632637, 0.0009929875445491103),
 'Precision@5': (0.027385746804412543, 0.001889158995464853),
 'Precision@50': (0.018182454911574157, 0.0006785318323012479),
 'R-Precision': (0.024283658684571233, 0.0017296679054643591),
 'Recall@10': (0.04343428093714735, 0.0033476891411462605),
 'Recall@15': (0.061021502223511596, 0.003952847924610231),
 'Recall@20': (0.07612098962109112, 0.004374829787968112),
 'Recall@5': (0.024784714124989775, 0.0026296878806785972),
 'Recall@50': (0.14855396919473482, 0.005

In [66]:
user_based_res750

{'MAP@10': (0.02946533840291501, 0.0018426257387044733),
 'MAP@15': (0.028044773277657187, 0.001551778177532631),
 'MAP@20': (0.026904716509549707, 0.001374140519084765),
 'MAP@5': (0.03168155022471254, 0.0024543725776863946),
 'MAP@50': (0.02314437756365863, 0.0009472330714907702),
 'NDCG': (0.07435153418521169, 0.0027046805809138774),
 'Precision@10': (0.025949921204692698, 0.0013506086267753565),
 'Precision@15': (0.024537442362691887, 0.0011328302371046938),
 'Precision@20': (0.02287690421992646, 0.0009855800602207856),
 'Precision@5': (0.02941691472596743, 0.0019174847810802893),
 'Precision@50': (0.018882857643144808, 0.0006858180679522091),
 'R-Precision': (0.028407526259754394, 0.0020410478125571893),
 'Recall@10': (0.04667103424061802, 0.0034260790631356467),
 'Recall@15': (0.06358967480608335, 0.003886318046289662),
 'Recall@20': (0.07867013620878373, 0.0043359254341710845),
 'Recall@5': (0.026691267104438997, 0.0025806494367606996),
 'Recall@50': (0.1587641406120259, 0.00594

In [52]:
user_based_res3000

{'MAP@10': (0.02974537442362692, 0.0018632660506386599),
 'MAP@15': (0.028267185158272534, 0.0015706152579902275),
 'MAP@20': (0.027139625024049967, 0.001390941617537062),
 'MAP@5': (0.031615011965213324, 0.0024798896544236657),
 'MAP@50': (0.02331262634689437, 0.0009567897797934154),
 'NDCG': (0.07487032972553509, 0.0027203133697012364),
 'Precision@10': (0.026912974960602343, 0.0013883630975664472),
 'Precision@15': (0.02451409560497286, 0.0011264885424188734),
 'Precision@20': (0.023095780073542294, 0.000994381981748592),
 'Precision@5': (0.02934687445281037, 0.0019247269552682598),
 'Precision@50': (0.019061460339695326, 0.0006936032876712233),
 'R-Precision': (0.027941033855279757, 0.0019820565641806955),
 'Recall@10': (0.048080167496898196, 0.0034748155450796217),
 'Recall@15': (0.06490903998957158, 0.003986161229945652),
 'Recall@20': (0.08057788371214346, 0.004414870896976773),
 'Recall@5': (0.02698460589678084, 0.002642747214667879),
 'Recall@50': (0.1591413191972895, 0.005895

In [60]:
user_based_res

{'MAP@10': (0.031344898316532005, 0.0019419718247370961),
 'MAP@15': (0.029866232135536987, 0.0016325577177255409),
 'MAP@20': (0.028656128642907317, 0.0014423906282694256),
 'MAP@5': (0.033376524835113525, 0.002590995553761898),
 'MAP@50': (0.024320038944187366, 0.0009847994668677619),
 'NDCG': (0.07721138347289283, 0.0028333472139054983),
 'Precision@10': (0.028383820696900722, 0.0014345802902826993),
 'Precision@15': (0.02613669526644487, 0.0011749008756399859),
 'Precision@20': (0.024505340570828225, 0.001029564088716077),
 'Precision@5': (0.030887760462265802, 0.002026027617556462),
 'Precision@50': (0.019355629486955, 0.0006955690047862636),
 'R-Precision': (0.028883717093828115, 0.002038446870856928),
 'Recall@10': (0.052008110922374604, 0.0036721612954724284),
 'Recall@15': (0.07063100178823213, 0.004204104035231979),
 'Recall@20': (0.08660429125621157, 0.004636511047404981),
 'Recall@5': (0.029146689761132605, 0.002806545567276444),
 'Recall@50': (0.15816837853808208, 0.005809

1. Other dataset
2. Better graph/table representation


In [73]:
def get_res(rtrain, rvalid, k=100, topK = 50, I_K = None, U_K = None, base = 0):
    """
    base = 0 means regular user_item KNN
    base = 1 means keyphrases-item_based similarity KNN
    base = 2 means keyphrases-user_based similarity KNN
    """
    if base == 0:
        similarity = train(rtrain)
    elif base == 1:
        similarity = train(I_K)
    elif base == 2:
        similarity = train(U_K)
    item_similarity_en = False
    if base == 1:
        item_similarity_en = True
    prediction_score = predict(rtrain, k, similarity, item_similarity_en= item_similarity_en)
    predict_res = prediction(prediction_score, topK, rtrain)
    res = evaluate(predict_res, rvalid)
    return res

In [75]:
I_K

<3668x3000 sparse matrix of type '<type 'numpy.int32'>'
	with 5101490 stored elements in Compressed Sparse Row format>

In [76]:
U_K

<6370x3000 sparse matrix of type '<type 'numpy.int32'>'
	with 444550 stored elements in Compressed Sparse Row format>

In [122]:
def get_res(similarity, rtrain, rvalid, k=100, topK = 50, I_K = None, U_K = None, base = 0):
    """
    base = 0 means regular user_item KNN
    base = 1 means keyphrases-item_based similarity KNN
    base = 2 means keyphrases-user_based similarity KNN
    """
    item_similarity_en = False
    if base == 1:
        item_similarity_en = True
    prediction_score = predict(rtrain, k, similarity, item_similarity_en= item_similarity_en)
    predict_res = prediction(prediction_score, topK, rtrain)
    res = evaluate(predict_res, rvalid)
    return res

In [146]:
def hyper_parameter_tuning(rtrain, rvalid, save_path, I_K, U_K):
    table_path = 'beer/tables/'
    try:
        df = load_dataframe_csv(table_path, save_path)
    except:
        df = pd.DataFrame(columns=['model', 'k', 'topK'])

    num_user = rtrain.shape[0]

    for algorithm in ['user_based']:
        
        if algorithm == 'user_item':
            similarity = train(rtrain)
        elif algorithm == 'item_based':
            similarity = train(I_K)
        elif algorithm == 'user_based':
            similarity = train(U_K)
            
        for k in [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 150, 200, 250, 300, 350, 400, 450, 500, 600, 700, 800, 900, 1000]:
            if algorithm == 'user_item':
                result =  get_res(similarity, rtrain,rvalid, k=k, base = 0)
                print "get result for k = ",k," , with algorithm = ", algorithm
            elif algorithm == 'item_based':
                result = get_res(similarity, rtrain,rvalid, k=k, I_K = I_K, base = 1)
                print "get result for k = ",k," , with algorithm = ", algorithm
            elif algorithm == 'user_based':
                result = get_res(similarity, rtrain,rvalid, k=k, U_K = U_K, base = 2)
                print "get result for k = ",k," , with algorithm = ", algorithm

            result_dict = {'model': algorithm, 'k': k}

            for name in result.keys():
                result_dict[name] = [round(result[name][0], 4), round(result[name][1], 4)]

            df = df.append(result_dict, ignore_index=True)

            save_dataframe_csv(df, table_path, save_path)

def save_dataframe_csv(df, path, name):
    df.to_csv(path+name, index=False)

In [147]:
hyper_parameter_tuning(rtrain, rvalid, 'knn_tunning_final_1500.csv', I_K1500, U_K1500)

100%|████████████████████████████████████████████████████████████████████████████| 6370/6370 [00:01<00:00, 5268.82it/s]


get result for k =  10  , with algorithm =  user_based


100%|████████████████████████████████████████████████████████████████████████████| 6370/6370 [00:01<00:00, 4953.34it/s]


get result for k =  20  , with algorithm =  user_based


100%|████████████████████████████████████████████████████████████████████████████| 6370/6370 [00:01<00:00, 5132.96it/s]


...

get result for k =  800  , with algorithm =  user_based


100%|████████████████████████████████████████████████████████████████████████████| 6370/6370 [00:01<00:00, 4892.47it/s]


get result for k =  900  , with algorithm =  user_based


100%|████████████████████████████████████████████████████████████████████████████| 6370/6370 [00:01<00:00, 4949.50it/s]


get result for k =  1000  , with algorithm =  user_based


# Test Result

### User-item result

In [129]:
rtest = load_npz('beer/Rtest.npz')

In [130]:
similarity = train(rtrain)
user_item_test_res = get_res(similarity, rtrain,rtest, k=200 , base = 0)

100%|████████████████████████████████████████████████████████████████████████████| 6370/6370 [00:01<00:00, 4873.76it/s]


In [132]:
user_item_test_res

{'MAP@10': (0.055037055095075574, 0.00271760155112745),
 'MAP@15': (0.051745667680821276, 0.00229974479769462),
 'MAP@20': (0.04911610731258947, 0.0020397178310452542),
 'MAP@5': (0.06045051194539249, 0.00358743056523425),
 'MAP@50': (0.04058976109960154, 0.0014208353232993188),
 'NDCG': (0.11061358430162982, 0.003160283681719818),
 'Precision@10': (0.04776450511945392, 0.00197653604717534),
 'Precision@15': (0.0432650739476678, 0.0016336711097877305),
 'Precision@20': (0.040051194539249144, 0.0014294610458038988),
 'Precision@5': (0.05430034129692834, 0.0027827016714734656),
 'Precision@50': (0.031003412969283276, 0.0009986026431826965),
 'R-Precision': (0.04616147106833989, 0.002303717415339684),
 'Recall@10': (0.07037424997438727, 0.0039006945367076992),
 'Recall@15': (0.09476025058335748, 0.004467342890288348),
 'Recall@20': (0.11499280080949816, 0.004854710107875195),
 'Recall@5': (0.039174533796154404, 0.0028903331435289827),
 'Recall@50': (0.20961257540339695, 0.0061101700902540

### Item_based

In [131]:
similarity = train(I_K5000)
item_based_test_res = get_res(similarity, rtrain, rtest, k=20 , I_K = I_K5000 , base = 1)

100%|████████████████████████████████████████████████████████████████████████████| 6370/6370 [00:01<00:00, 4666.67it/s]


In [133]:
item_based_test_res

{'MAP@10': (0.022312415352944363, 0.0016645119503467458),
 'MAP@15': (0.020760076638062987, 0.0013902801832968463),
 'MAP@20': (0.019681531591873785, 0.0012239746277176403),
 'MAP@5': (0.024571103526734927, 0.0022296530333048496),
 'MAP@50': (0.016267408710191472, 0.0008264043406741288),
 'NDCG': (0.04288493444680673, 0.0020306935982967473),
 'Precision@10': (0.01861774744027304, 0.0011760285178246187),
 'Precision@15': (0.017064846416382253, 0.0009640787637895016),
 'Precision@20': (0.015955631399317407, 0.0008507705336377911),
 'Precision@5': (0.022969283276450512, 0.00172364733170156),
 'Precision@50': (0.012580204778156997, 0.0005672840933801791),
 'R-Precision': (0.017466114041808117, 0.0014085103984669777),
 'Recall@10': (0.025981601839728127, 0.002409947202481572),
 'Recall@15': (0.034509479591048456, 0.0027058987502942195),
 'Recall@20': (0.04287631515663236, 0.0030315121081121673),
 'Recall@5': (0.016478250732903512, 0.0019431723508946126),
 'Recall@50': (0.08058585111864981, 

### User_based

In [134]:
similarity = train(U_K5000)
user_based_test_res = get_res(similarity, rtrain, rtest, k=300 , U_K = U_K5000 , base = 2)

100%|████████████████████████████████████████████████████████████████████████████| 6370/6370 [00:01<00:00, 4976.56it/s]


In [136]:
user_based_test_res = get_res(similarity, rtrain, rtest, k=200 , U_K = U_K5000 , base = 2)

100%|████████████████████████████████████████████████████████████████████████████| 6370/6370 [00:01<00:00, 5091.93it/s]


In [135]:
user_based_test_res

{'MAP@10': (0.03732192968199794, 0.002190998285361629),
 'MAP@15': (0.03540604781389765, 0.0018550877020044605),
 'MAP@20': (0.03401433143888454, 0.001649645956359752),
 'MAP@5': (0.04009158134243458, 0.0028868989007371213),
 'MAP@50': (0.02915147391915201, 0.0011513328248138103),
 'NDCG': (0.08112615313012567, 0.002671530721609616),
 'Precision@10': (0.03320819112627986, 0.0016160826098981346),
 'Precision@15': (0.030898748577929465, 0.0013541735774195904),
 'Precision@20': (0.029215017064846416, 0.0011951077880419058),
 'Precision@5': (0.03747440273037543, 0.0022849286183576727),
 'Precision@50': (0.02388737201365188, 0.0008305650195755597),
 'R-Precision': (0.0317810013374843, 0.0019073235408918732),
 'Recall@10': (0.04585570588949454, 0.0031493405930743813),
 'Recall@15': (0.0642284711656551, 0.0037279286209226923),
 'Recall@20': (0.07934619698117001, 0.004060931754391117),
 'Recall@5': (0.02554002106144808, 0.002333864577671177),
 'Recall@50': (0.16191686996875107, 0.0055038677756

In [148]:
similarity = train(U_K1500)
user_based_test_res = get_res(similarity, rtrain, rtest, k=300 , U_K = U_K5000 , base = 2)

100%|████████████████████████████████████████████████████████████████████████████| 6370/6370 [00:01<00:00, 4636.10it/s]


In [151]:
user_based_test_res

{'MAP@10': (0.037076520938295686, 0.0021836381227901313),
 'MAP@15': (0.03519383378598396, 0.0018524455232287705),
 'MAP@20': (0.033752377268455466, 0.0016467131664474927),
 'MAP@5': (0.03970079635949943, 0.0028627113837744406),
 'MAP@50': (0.02901247820595199, 0.0011484030056434567),
 'NDCG': (0.0809328726602833, 0.0026708968864754845),
 'Precision@10': (0.03290102389078498, 0.0016083906056846586),
 'Precision@15': (0.030625711035267348, 0.001352170620200592),
 'Precision@20': (0.02871160409556314, 0.0011796532304680235),
 'Precision@5': (0.037440273037542667, 0.0022969449504500407),
 'Precision@50': (0.023918088737201363, 0.000830700999192574),
 'R-Precision': (0.03140600834368189, 0.0019051571701536505),
 'Recall@10': (0.046410763095268745, 0.0032310746391458513),
 'Recall@15': (0.06350003836979959, 0.003699626763247859),
 'Recall@20': (0.07840436401688601, 0.00406143081888389),
 'Recall@5': (0.02569159156512268, 0.0023676209099114825),
 'Recall@50': (0.16196184143576622, 0.00550025